<a href="https://colab.research.google.com/github/EmelyanovAndreyNSK/PythonProjects/blob/master/ensemble_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.1/874.1 kB 28.6 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import numpy as np
import cv2
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import requests

import zipfile

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# URL для загрузки изображений
image_url = "http://images.cocodataset.org/zips/val2017.zip"

# Загрузка изображений
response = requests.get(image_url)
with open("val2017.zip", "wb") as f:
    f.write(response.content)

In [ ]:
# URL для загрузки аннотаций
annotation_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

# Загрузка аннотаций
response = requests.get(annotation_url)
with open("annotations_trainval2017.zip", "wb") as f:
    f.write(response.content)

In [ ]:
# Распаковка аннотаций
with zipfile.ZipFile("annotations_trainval2017.zip", "r") as zip_ref:
    zip_ref.extractall("annotations")

# Распаковка изображений
with zipfile.ZipFile("val2017.zip", "r") as zip_ref:
    zip_ref.extractall("images")

In [ ]:
# Load a model
# Replace 'path/to/model1.pt', 'path/to/model2.pt', etc. with the paths to your trained models
model1 = YOLO('yolov5n.pt')  # load a custom model
model2 = YOLO('yolov8n.pt')  # load another custom model
# Add more models as needed

# Define the weights for each model
weights = np.array([1.0, 1.0])  # replace with your weights
# The weights should sum to 1 and should be in the same order as the models

# Normalize the weights so they sum to 1
#weights /= np.sum(weights)


PRO TIP 💡 Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 5.31M/5.31M [00:00<00:00, 200MB/s]


100%|██████████| 6.25M/6.25M [00:00<00:00, 240MB/s]


In [ ]:
# Define a function to make predictions with all models
def predict_ensemble(image_path):
    # Make predictions with each model
    results1 = model1.predict(source=image_path)
    results2 = model2.predict(source=image_path)
    # Add more predictions as needed

    # Combine the results with the weights
    # This is a very basic example and may not work well
    # In a real-world scenario, you might want to use a more sophisticated method to combine the results
    #combined_results = results1 * weights[0] + results2 * weights[1]
    combined_results = results1 + results2

    # Apply non-maximum suppression to the combined results
    # This assumes that the results are in the format [x1, y1, x2, y2, confidence]
    # You may need to adjust this depending on the format of your results
    print(combined_results)
    combined_results = np.array(combined_results)

    pick = cv2.dnn.NMSBoxes(combined_results[:, :4], combined_results[:, 4], 0.5, 0.5)
    combined_results = combined_results[pick]

    return combined_results

In [ ]:
# Load the COCO validation set
coco = COCO('/content/annotations/annotations/instances_val2017.json')

# Get the IDs of the first 500 images in the validation set
image_ids = coco.getImgIds()[:500]

# Make predictions on the validation set and format the results in COCO format
coco_results = []
for image_id in image_ids:
    # Get the image path
    image_info = coco.loadImgs(image_id)[0]
    image_path = '/content/images/val2017/' + image_info['file_name']

    # Make predictions on the image
    results = predict_ensemble(image_path)

    # Format the results in COCO format
    for result in results:
        x1, y1, x2, y2, score = result
        w = x2 - x1
        h = y2 - y1
        coco_results.append({
            'image_id': image_id,
            'category_id': 1,  # replace with the actual category ID
            'bbox': [x1, y1, w, h],
            'score': score
        })

# Compute the mAP metric
coco_true = coco.loadRes('/content/annotations/annotations/instances_val2017.json')
coco_pred = coco.loadRes(coco_results)
coco_eval = COCOeval(coco_true, coco_pred, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()